In [21]:
import math
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
from catboost import CatBoostClassifier, CatBoostRegressor
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.metrics import RMSE
from pytorch_forecasting.models import TemporalFusionTransformer
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.data import DataLoader, Dataset

warnings.filterwarnings("ignore")

In [6]:
df = pd.read_csv("../../data/frames_errors.csv", header=None)
df.columns = [
    "block_id",
    "frame_idx",
    "E_mu_Z",
    "E_mu_phys_est",
    "E_mu_X",
    "E_nu1_X",
    "E_nu2_X",
    "E_nu1_Z",
    "E_nu2_Z",
    "N_mu_X",
    "M_mu_XX",
    "M_mu_XZ",
    "M_mu_X",
    "N_mu_Z",
    "M_mu_ZZ",
    "M_mu_Z",
    "N_nu1_X",
    "M_nu1_XX",
    "M_nu1_XZ",
    "M_nu1_X",
    "N_nu1_Z",
    "M_nu1_ZZ",
    "M_nu1_Z",
    "N_nu2_X",
    "M_nu2_XX",
    "M_nu2_XZ",
    "M_nu2_X",
    "N_nu2_Z",
    "M_nu2_ZZ",
    "M_nu2_Z",
    "nTot",
    "bayesImVoltage",
    "opticalPower",
    "polarizerVoltages[0]",
    "polarizerVoltages[1]",
    "polarizerVoltages[2]",
    "polarizerVoltages[3]",
    "temp_1",
    "biasVoltage_1",
    "temp_2",
    "biasVoltage_2",
    "synErr",
    "N_EC_rounds",
    "maintenance_flag",
    "estimator_name",
    "f_EC",
    "E_mu_Z_est",
    "R",
    "s",
    "p",
]

In [7]:
id_features = [
    "block_id",
    "frame_idx",
]

phys_gt_features = [
    "E_mu_Z",
    "E_mu_X",
    "E_nu1_X",
    "E_nu2_X",
    "E_nu1_Z",
    "E_nu2_Z",
    "N_mu_X",
    "M_mu_XX",
    "M_mu_XZ",
    "M_mu_X",
    "N_mu_Z",
    "M_mu_ZZ",
    "M_mu_Z",
    "N_nu1_X",
    "M_nu1_XX",
    "M_nu1_XZ",
    "M_nu1_X",
    "N_nu1_Z",
    "M_nu1_ZZ",
    "M_nu1_Z",
    "N_nu2_X",
    "M_nu2_XX",
    "M_nu2_XZ",
    "M_nu2_X",
    "N_nu2_Z",
    "M_nu2_ZZ",
    "M_nu2_Z",
]

phys_features = [
    "bayesImVoltage",
    "opticalPower",
    "polarizerVoltages[0]",
    "polarizerVoltages[1]",
    "polarizerVoltages[2]",
    "polarizerVoltages[3]",
    "temp_1",
    "biasVoltage_1",
    "temp_2",
    "biasVoltage_2",
]

est_features = [
    # "E_mu_phys_est",
    "E_mu_Z_est",
    "R",
    "s",
    "p",
]

proxy_features = [
    "N_EC_rounds",
    # "f_EC",
]

df = df[id_features + phys_gt_features + phys_features + est_features + proxy_features]

In [8]:
def lag_features(df, features, lag):
    df = df.copy()
    lagged_features = []

    for feature in features:
        for l in lag:
            df[f"{feature}_lag_{l}"] = df.groupby("block_id")[feature].shift(l)
            lagged_features.append(f"{feature}_lag_{l}")

    df = df.bfill().ffill()
    return df, lagged_features

In [9]:
features_to_create = phys_gt_features + phys_features + est_features
df, lagged_features = lag_features(df, features_to_create, lag=[1, 2, 3, 5, 10])

In [10]:
# Test (Val)
start_idx = df[(df["block_id"] == 1489460492) & (df["frame_idx"] == 99)].index[0]
end_idx = df[(df["block_id"] == 1840064900) & (df["frame_idx"] == 101)].index[0]

test_df = df.loc[start_idx:end_idx].copy()
assert len(test_df) == 2000, "Test (Val) set size is not 2000 rows"

# Train
all_block_ids = df["block_id"].unique()
train_blocks = [bid for bid in all_block_ids if bid not in test_df["block_id"].values]

train_df = df[df["block_id"].isin(train_blocks)]
train_df = train_df[(train_df["N_EC_rounds"] <= 1)]

print(f"Train: {len(train_blocks)}")

Train: 817


In [11]:
feature_cols = phys_features + lagged_features + ["E_mu_Z_est", "R"]
target_col = "s"

In [13]:
scaler = StandardScaler()
scaler.fit(train_df[feature_cols])

train_df.loc[:, feature_cols] = scaler.transform(train_df[feature_cols])
test_df.loc[:, feature_cols] = scaler.transform(test_df[feature_cols])

train_df.loc[:, feature_cols] = train_df[feature_cols].astype(np.float32)
test_df.loc[:, feature_cols] = test_df[feature_cols].astype(np.float32)

In [32]:
# Копии датасетов для TFT
train_df_tft = train_df.copy()
test_df_tft = test_df.copy()

# Индекс времени
train_df_tft["time_idx"] = train_df_tft["frame_idx"].astype(int)
test_df_tft["time_idx"] = test_df_tft["frame_idx"].astype(int)

# Конфигурация окон
max_encoder_length = 60  # длина "прошлого"
max_prediction_length = 1  # прогноз на 1 шаг вперёд

# Фичи
time_varying_known_reals = feature_cols
time_varying_unknown_reals = [target_col]  # предсказываемая цель

training = TimeSeriesDataSet(
    train_df_tft,
    time_idx="time_idx",
    target=target_col,
    group_ids=["block_id"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=time_varying_known_reals,
    time_varying_unknown_reals=time_varying_unknown_reals,
    target_normalizer=None,
    allow_missing_timesteps=True,
)

validation = TimeSeriesDataSet.from_dataset(
    training, test_df_tft, predict=True, stop_randomization=True
)

batch_size = 64
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=4)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=4)

In [33]:
from lightning.pytorch import Trainer

In [34]:
pl.seed_everything(42, workers=True)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=3e-4,
    hidden_size=128,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=64,
    output_size=1,  # точечный прогноз
    loss=RMSE(),  # RMSE как функция потерь
    log_interval=50,
    reduce_on_plateau_patience=4,
)

trainer = Trainer(
    max_epochs=30,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    devices=1,
    gradient_clip_val=1.0,
    enable_checkpointing=False,
    logger=False,
)

trainer.fit(tft, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | RMSE                            | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 0      | train
3  | prescalers                         | ModuleDict                      | 27.9 K | train
4  | static_variable_selection          | VariableSelectionNetwork        | 0      | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 7.5 M  | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 7.5 M  | train
7

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
pred_batches = trainer.predict(tft, dataloaders=val_dataloader, return_x=True)

# pred_batches — список из (y_hat, x). y_hat: (B, max_prediction_length, 1)
y_pred = torch.cat([p[0] for p in pred_batches], dim=0).squeeze(-1).cpu().numpy()
y_true = torch.cat([x["decoder_target"] for _, x in pred_batches], dim=0).squeeze(-1).cpu().numpy()

val_mse = float(np.mean((y_pred - y_true) ** 2))
val_rmse = float(np.sqrt(val_mse))
print(f"TFT val MSE: {val_mse:.6f} | RMSE: {val_rmse:.6f}")

AttributeError: module 'pytorch_lightning' has no attribute '__version_'

In [27]:
print(type(tft))

<class 'pytorch_forecasting.models.temporal_fusion_transformer._tft.TemporalFusionTransformer'>
